exactly in the same way than in the past notebook, if you dont have any of these packages installed, you will need to install them first. 
cmd command or command terminal
'pip install  name of the package'


Now, this code is splied into 3 parts: the calling for all the necessary packages. The path setup and the rest of the process.
I recomend you tu run each part separately to know if you have any issues and solve them.

In [16]:
import logging 
import os 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time
import json
from selenium.webdriver.common.keys import Keys

These are the paths with the routes to:
1. Path where you located the cvs file with the links. this is the same worked in the previous notebook.
2. Path where you want to download all the files. 
As a recomendation, I usually save all my files in the "raw data" folder, this in order to push all the files directly to the github repo.

In [17]:
csv_path = '/home/alrier/Documentos/GitHub/SoftSkillsLatam/Costa_Rica/URLs.csv' #path where you located the cvs file with the links.
file_path = '/home/alrier/Documentos/GitHub/SoftSkillsLatam/Costa_Rica/' # path to download the university files.
df = pd.read_csv(csv_path)

Once you have the paths seted up, you can run the following lines.

This code will open a google chrome web browser using selenium, will loop through the URL columns in the csv file deined in the first path, then will open each link and look for the keywords = ["Program", "Master", "Doctor", "PhD", "Ph.D."]. You can always change it if you consider it necessary. Now, if the code finds any of all of the keywords in the text, then it will print and wownload the document in the second path defined previously. 

the codu wil create a security copy of the original csv file in the path defined  here: new_csv_path = "/home/alrier/Documentos/canada/USA_modified.csv" So, better you change this path to anyone you want.

As a setup recomendations, some webpages have javascript interactive buttons. It force the browser to take more time in the process. If you see that the code colapse becouse of the timing: WebDriverWait(driver, 3) --> this is the timer for the code, the only thing you need to do is, modify this variable to the number of seconds you want the code to wait until the browser is ready.

In [18]:
# Chrome options
options = webdriver.ChromeOptions()
options.add_argument("--kiosk-printing")
settings = {
    "recentDestinations": [{
        "id": "Save as PDF",
        "origin": "local",
        "account": "",
    }],
    "selectedDestinationId": "Save as PDF",
    "version": 2
}
prefs = {'printing.print_preview_sticky_settings.appState': json.dumps(settings),
         "savefile.default_directory": file_path, }
options.add_experimental_option('prefs', prefs)
# Loop through URLs
for index, row in df.iterrows():
    url = row['URL']
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    try:
        dynamic_element = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, "//button[contains(@class,'animated')]"))
        )
        dynamic_element.click()
        WebDriverWait(driver, 3).until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
    except:
        pass
    keywords = ["master", "Master", "MSc", "PhD", "phd", "Doctorado", "doctorado", "maestría"]
    for keyword in keywords:
        if driver.find_elements(By.XPATH, f"//*[contains(text(), '{keyword}')]"):
            driver.execute_script('window.print();')
            break
    driver.quit()
    # Delete the URL from the csv file after each iteration.
    df.at[index, 'URL'] = ""
# Save the modified dataframe in a new CSV file
new_csv_path = '/home/alrier/Documentos/GitHub/SoftSkillsLatam/Costa_Rica/URLs.csv' 
df.to_csv(new_csv_path, index=False)
# Rename each PDF in the "pdf_files_temp" folder.
pdf_temp_path = os.path.join(file_path, "pdf_files_temp")
for file in os.listdir(pdf_temp_path):
    os.rename(os.path.join(pdf_temp_path, file), os.path.join(file_path, "USA_" + file))
# Delete the "pdf_files_temp" after the process is completed.
os.rmdir(pdf_temp_path)
print('Completed!.') #if successfully completed.

Completed!.
